In [1]:
import pandas as pd
import numpy as np

In [2]:
from tqdm import tqdm
tqdm.pandas()

In [31]:
import operator 
import re

In [3]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [4]:
print(df_train.shape)
print(df_test.shape)

(1306122, 3)
(56370, 2)


In [12]:
df_train.head()

,question_text,target
0,how did quebec nationalists see their province...,0
1,"do you have an adopted dog , how would you en...",0
2,why does velocity affect time ? does velocity...,0
3,how did otto von guericke used the magdeburg h...,0
4,can i convert montra helicon d to a mountain b...,0


In [13]:
df_test.head()

,question_text
0,my voice range is a2 - c5 . my chest voice go...
1,how much does a tutor earn in bangalore ?
2,what are the best made pocket knives under $ ...
3,why would they add a hypothetical scenario tha...
4,what is the dresscode for techmahindra fresher...


In [7]:
## Now removing the qid from test and train data
##but storing the test qid for submission

In [8]:
Y_QID=df_test['qid']

In [9]:
del df_test['qid']
del df_train['qid']

In [10]:
##Changing the words in lower case
df_train["question_text"] = df_train["question_text"].str.lower()
df_test["question_text"] = df_test["question_text"].str.lower()

In [11]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):

    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


df_train["question_text"] = df_train["question_text"].apply(lambda x: clean_text(x))
df_test["question_text"] = df_test["question_text"].apply(lambda x: clean_text(x))

In [14]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [15]:
sentences = df_train["question_text"].progress_apply(lambda x: x.split()).values
vocab = build_vocab(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

100%|██████████| 1306122/1306122 [00:04<00:00, 287047.80it/s]

{'how': 290379, 'did': 41736, 'quebec': 168, 'nationalists': 151, 'see': 9710}


In [18]:
## Embedding 

In [23]:
def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    if file == '/Users/shivangisingh/Downloads/Quora/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index

In [26]:
glove = '/Users/shivangisingh/Downloads/Quora/embeddings/glove.840B.300d/glove.840B.300d.txt'
paragram =  '/Users/shivangisingh/Downloads/Quora/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
wiki_news = '/Users/shivangisingh/Downloads/Quora/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'

In [27]:
print("Extracting GloVe embedding")
embed_glove = load_embed(glove)
print("Extracting Paragram embedding")
embed_paragram = load_embed(paragram)
print("Extracting FastText embedding")
embed_fasttext = load_embed(wiki_news)

Extracting GloVe embedding
Extracting Paragram embedding
Extracting FastText embedding


In [29]:
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in tqdm(vocab.keys()):
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

In [32]:
print("Glove : ")
oov_glove = check_coverage(vocab, embed_glove)
print("Paragram : ")
oov_paragram = check_coverage(vocab, embed_paragram)
print("FastText : ")
oov_fasttext = check_coverage(vocab, embed_fasttext)

 65%|██████▌   | 127649/195993 [00:00<00:00, 649031.32it/s]

Glove : 


 14%|█▍        | 27572/195993 [00:00<00:00, 275714.35it/s]

Found embeddings for 63.28% of vocab
Found embeddings for  99.20% of all text
Paragram : 


 15%|█▌        | 30198/195993 [00:00<00:01, 144404.27it/s]

Found embeddings for 73.91% of vocab
Found embeddings for  99.45% of all text
FastText : 


100%|██████████| 195993/195993 [00:00<00:00, 362500.29it/s]

Found embeddings for 48.50% of vocab
Found embeddings for  98.81% of all text


In [35]:
oov_glove[1:10]

[('”', 3911),
 ('“', 3871),
 ('…', 1928),
 ('‘', 874),
 ('é', 864),
 ('quorans', 858),
 ('brexit', 524),
 ('cryptocurrencies', 499),
 ('√', 395)]

In [37]:
'&' in oov_paragram

False

In [39]:
oov_paragram[1:10]

[('”', 3911),
 ('“', 3871),
 ('…', 1928),
 ('‘', 874),
 ('é', 864),
 ('quorans', 858),
 ('brexit', 524),
 ('cryptocurrencies', 499),
 ('√', 395)]

In [52]:
def clean_embedding(embedding,punct):
    returnableArr = []
    for p in embedding:
        if not p[0] in punct:
            returnableArr.append(p)
    return returnableArr        

In [53]:
new_embedding=clean_embedding(oov_paragram,puncts)

In [54]:
new_embedding[1:10]

[('brexit', 524),
 ('cryptocurrencies', 499),
 ('redmi', 383),
 ('coinbase', 149),
 ('oneplus', 139),
 ('uceed', 123),
 ('demonetisation', 115),
 ('bhakts', 115),
 ('upwork', 111)]

In [56]:
print({k: vocab[k] for k in list(vocab)[:10]})

{'how': 290379, 'did': 41736, 'quebec': 168, 'nationalists': 151, 'see': 9710, 'their': 34975, 'province': 191, 'as': 60532, 'a': 411632, 'nation': 1525}


In [62]:
len(new_embedding)

51078

In [ ]:
##Taking the embedding size=300

In [63]:
##Creating the embeeding vector

In [64]:
def embedding_dict(embedding):
    embed_dict={}
    for i in embedding:
        embed_dict[i[0]]=i[1]
    return embed_dict    

In [65]:
embedding_dic=embedding_dict(new_embedding)

In [69]:
len(embedding_dic.keys())

51078

In [61]:
sentences[3]

['how',
 'did',
 'otto',
 'von',
 'guericke',
 'used',
 'the',
 'magdeburg',
 'hemispheres',
 '?']